# KRX 데이터 조회 - 전일급등주
- description: 가즈아
- layout: post
- categories: [경제]
- toc: true

In [1]:
#hide
!pip install pandas pandasql xlrd

  Created wheel for pandasql: filename=pandasql-0.7.3-cp36-none-any.whl size=26819 sha256=52fb20c8d23e2caad374f3c00a3f7c3c13a29b6504c5d5f863ddb34a98838bc2
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [3]:
# hide
import requests
import pandas as pd
from pandas.tseries.offsets import BDay
from io import StringIO
from io import BytesIO
import json
import datetime

왜 이렇게 만들었는지 모르겠는데 KRX는 한번 꼬아서 다운로드 받는다. 이러면 내가 못할 줄 알고?

In [4]:
def downloadKrxData(cd):
    resp = requests.post(
        'http://file.krx.co.kr/download.jspx', 
        headers={
            'User-Agent': 'Mozilla/5.0', 
            'Referer': 'http://marketdata.krx.co.kr/',
        }, 
        params={'code': cd},
    )
    tp = resp.headers['Content-Disposition'].split('.')[1]
    if tp == 'csv':
        return pd.read_csv(StringIO(resp.text))
    if tp == 'xls':
        return pd.read_excel(BytesIO(resp.content))

## KRX 휴장일을 반영한 전영업일 
- KRX 01023 휴장일 http://marketdata.krx.co.kr/mdi#document=01100305

In [5]:
def closingDays(year):
    resp = requests.get(
        'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx', 
        headers={'User-Agent': 'Mozilla/5.0'}, 
        params={
            'name': 'fileDown',
            'filetype': 'xls',
            'url': 'MKD/01/0110/01100305/mkd01100305_01',
            'search_bas_yy': year,
            'gridTp': 'KRX',
            'pagePath': '/contents/MKD/01/0110/01100305/MKD01100305.jsp',
        },
    )
    return downloadKrxData(resp.text)['일자 및 요일']

def getPreviousBizDay(today):
    pday = datetime.datetime.strptime(today, '%Y%m%d')
    yyyy = pday.year
    cds = closingDays(yyyy)

    while True:
        pday = pday - BDay(1)
        if yyyy != pday.year:
            yyyy = pday.year
            cds = pd.concat([cds, closingDays(yyyy)])
        if pday.strftime("%Y-%m-%d") not in cds.values:
            return pday.strftime("%Y%m%d")

## 전일 등락
- KRX 80037 전체종목 등락률 http://marketdata.krx.co.kr/mdi#document=13020102

In [6]:
def soaredStocks(dt):
    print(dt)
    resp = requests.get(
        'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx', 
        headers={'User-Agent': 'Mozilla/5.0'}, 
        params={
            'name': 'fileDown',
            'filetype': 'csv',
            'url': 'MKD/13/1302/13020102/mkd13020102',
            'ind_tp': 'ALL',
            'period_strt_dd': dt,
            'period_end_dd': dt,
            'pagePath': '/contents/MKD/13/1302/13020102/MKD13020102.jsp',
        },
    )
    return downloadKrxData(resp.text)

In [7]:
# hide
pd.set_option('display.max.colwidth', 150)

## 전 영업일 15% 이상 오른 종목

In [8]:
dt = getPreviousBizDay(datetime.datetime.now().strftime('%Y%m%d'))
# dt = '20201026'
df_soarings = soaredStocks(dt)

20201103


In [9]:
# hide
df_15 = df_soarings[df_soarings.등락률 >= 15].sort_values('등락률', ascending=False)
df_15['네이버주식'] = 'https://finance.naver.com/item/main.nhn?code=' + df_15['종목코드']
df_15

,종목코드,종목명,시작일기준가(원),종료일종가(원),대비,등락률,거래량(주),거래대금(원),네이버주식
2406,323990,박셀바이오,"54,000","70,200","16,200",30.00,"5,214,897","339,229,618,900",https://finance.naver.com/item/main.nhn?code=323990
1006,045890,GV,507,659,152,29.98,"47,495,448","29,845,022,842",https://finance.naver.com/item/main.nhn?code=045890
1048,049630,재영솔루텍,691,898,207,29.96,"22,034,422","18,700,106,079",https://finance.naver.com/item/main.nhn?code=049630
2045,217600,켐온,"3,555","4,620","1,065",29.96,"90,597,008","384,634,638,710",https://finance.naver.com/item/main.nhn?code=217600
2201,258790,소프트캠프,"2,615","3,395",780,29.83,"4,041,761","12,822,923,030",https://finance.naver.com/item/main.nhn?code=258790
272,004835,덕성우,"11,950","15,500","3,550",29.71,"2,897,423","43,156,022,900",https://finance.naver.com/item/main.nhn?code=004835
346,006280,녹십자,"301,500","378,000","76,500",25.37,"2,859,692","990,452,629,500",https://finance.naver.com/item/main.nhn?code=006280
2245,269620,시스웍,"3,520","4,295",775,22.02,"67,408,341","286,541,994,430",https://finance.naver.com/item/main.nhn?code=269620
1150,057500,SKC 솔믹스,"5,000","6,060","1,060",21.20,"3,797,946","23,918,655,850",https://finance.naver.com/item/main.nhn?code=057500
2346,304840,피플바이오,"38,750","46,600","7,850",20.26,"9,420,213","426,908,689,850",https://finance.naver.com/item/main.nhn?code=304840


In [10]:
import pandasql as ps
ps.sqldf('''
    select a.*,
           'https://finance.naver.com/item/main.nhn?code=' || a.종목코드 as '네이버주식'
      from df_soarings a
     where 등락률 >= 15
  order by 등락률 desc
''')

,종목코드,종목명,시작일기준가(원),종료일종가(원),대비,등락률,거래량(주),거래대금(원),네이버주식
0,323990,박셀바이오,"54,000","70,200","16,200",30.00,"5,214,897","339,229,618,900",https://finance.naver.com/item/main.nhn?code=323990
1,045890,GV,507,659,152,29.98,"47,495,448","29,845,022,842",https://finance.naver.com/item/main.nhn?code=045890
2,049630,재영솔루텍,691,898,207,29.96,"22,034,422","18,700,106,079",https://finance.naver.com/item/main.nhn?code=049630
3,217600,켐온,"3,555","4,620","1,065",29.96,"90,597,008","384,634,638,710",https://finance.naver.com/item/main.nhn?code=217600
4,258790,소프트캠프,"2,615","3,395",780,29.83,"4,041,761","12,822,923,030",https://finance.naver.com/item/main.nhn?code=258790
5,004835,덕성우,"11,950","15,500","3,550",29.71,"2,897,423","43,156,022,900",https://finance.naver.com/item/main.nhn?code=004835
6,006280,녹십자,"301,500","378,000","76,500",25.37,"2,859,692","990,452,629,500",https://finance.naver.com/item/main.nhn?code=006280
7,269620,시스웍,"3,520","4,295",775,22.02,"67,408,341","286,541,994,430",https://finance.naver.com/item/main.nhn?code=269620
8,057500,SKC 솔믹스,"5,000","6,060","1,060",21.20,"3,797,946","23,918,655,850",https://finance.naver.com/item/main.nhn?code=057500
9,304840,피플바이오,"38,750","46,600","7,850",20.26,"9,420,213","426,908,689,850",https://finance.naver.com/item/main.nhn?code=304840
